In [14]:
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
import galsim
import cv2 as cv
import tensorflow as tf

%matplotlib inline
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

In [ ]:
tf.version.VERSION

In [ ]:
dat = np.load('Data/Cat-SFG-17-01-2020.npz')
dat.files

In [ ]:
dat['listgal'].shape

In [ ]:
img1 = np.abs(fits.open('Data/psf2asec-96.psf.fits')[0].data[0,0,:,:])

### Loading PSF & SFG

In [ ]:
gal = dat['listgal'][0,:,:]
psf = galsim.Image(img1, copy=True).array

psf /= np.sum(psf)

In [ ]:
fig = plt.figure(figsize=(12,12))

plt.subplot(221)
plt.imshow(gal)
plt.colorbar(shrink=0.825)
plt.title('Ground Truth')

plt.subplot(222)
plt.imshow(np.log10(gal))
plt.colorbar(shrink=0.825)
plt.title('Log (Ground Truth)')

plt.subplot(223)
plt.imshow(psf)
plt.colorbar(shrink=0.825)
plt.title('Given PSF')

plt.subplot(224)
plt.imshow(np.log10(psf))
plt.colorbar(shrink=0.825)
plt.title('Log (Given PSF)')

plt.show()

### FFT(Ground Truth) + Gaussian Noise

In [ ]:
gal_fft = np.fft.fftshift(np.fft.fft2(gal))   
psf_fft = np.fft.fftshift(np.fft.fft2(psf))      

# SNR(x) = l2_norm(x)/sigma_noise 

SNR = 1.0e2                                                       # Signal to noise ratio in Fourier domain
sigma_noise = np.linalg.norm(gal_fft)/SNR                         # Standard deviation of noise

gal_fft_GSO = galsim.Image(gal_fft, copy=True)                    # Stored as GSO
gal_fft_GSO.addNoise(galsim.GaussianNoise(sigma = sigma_noise)) 


fig = plt.figure(figsize=(12,12))

plt.subplot(121)
plt.imshow(np.abs(gal_fft_GSO.array))
plt.colorbar(shrink=0.375)
plt.title('FFT(Ground Truth) + Noise')

plt.subplot(122)
plt.imshow(np.log10(np.abs(gal_fft_GSO.array)))
plt.colorbar(shrink=0.375)
plt.title('Log (FFT(Ground Truth) + Noise)')

plt.show()

### Convolve with PSF

In [ ]:
pad = 20        # Padding to remove aliasing

conv_padded = np.pad(gal_fft_GSO.array, pad) * np.pad(psf_fft, pad)
conv = conv_padded[pad:conv_padded.shape[0]-pad, pad:conv_padded.shape[1]-pad]

final_gal = np.real(np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(conv))))  # IFFT to get back to brightness domain
final_gal = final_gal * (np.sum(gal)/np.sum(final_gal))                     # Normalisation


fig = plt.figure(figsize=(12,12))

plt.subplot(121)
plt.imshow(final_gal)
plt.colorbar(shrink=0.375)
plt.title('Conv (Ground Truth, PSF)')

plt.subplot(122)
plt.imshow(np.log10(final_gal))
plt.colorbar(shrink=0.375)
plt.title('Log (Conv (Ground Truth, PSF))')

plt.show()

# ADMM in TensorFlow

In [ ]:
# Convert arrays to tensors

gal_tf = tf.cast(gal, dtype='complex128')
final_gal_tf = tf.cast(final_gal, dtype='complex128')
psf_tf = tf.cast(psf, dtype='complex128')

In [ ]:
# Utility Functions

def fftconvolve(image, kernel):
   
    x = tf.signal.fftshift(tf.signal.fft2d(image))
    y = tf.signal.fftshift(tf.signal.fft2d(kernel))
    result = tf.math.real(tf.signal.fftshift(tf.signal.ifft2d(tf.signal.ifftshift(x * y))))

    return tf.cast(result, dtype='complex128')


def fft(data):

    return ( tf.convert_to_tensor(1.0/data.get_shape().as_list()[0], dtype='complex128') * 
             tf.signal.fftshift(tf.signal.fft2d(tf.signal.ifftshift(data))) )


def ifft(data):

    return ( tf.convert_to_tensor(data.get_shape().as_list()[0], dtype='complex128') *
             tf.signal.fftshift(tf.signal.ifft2d(tf.signal.ifftshift(data))) )


def soft_thresh(data, threshold):

    return tf.cast(tf.math.maximum((1.0 - threshold/tf.math.abs(data)), 0.0), dtype='complex128') * data


def nmse(signal_1, signal_2):

    return tf.keras.backend.get_value(tf.norm(signal_2 - signal_1)**2 / tf.norm(signal_1)**2)

In [ ]:
# Projection - Enforce non-negative values
def proj(xi):
    
    return tf.cast(tf.math.maximum(tf.math.real(xi), 0.0), dtype='complex128')

 
# H operator
def H(data, psf):
    
    return fftconvolve(data, psf)


# H transpose operator
def Ht(data, psf):
    
    return fftconvolve(data, tf.reverse(tf.reverse(psf, axis=[0]), axis=[1]))      # rotate by 180


# The gradient
def grad(y, x_rec, psf):

    return ifft(Ht(H(fft(x_rec), psf) - y, psf))


# The cost function
def cost_func(y, x_rec, z, mu, rho, psf, var=1.0):

    cost_val = ( tf.cast(0.5, dtype='complex128') * rho * tf.norm((z - x_rec))**2  -
                 tf.tensordot(mu, (z - x_rec), 1)  +
                 tf.cast(0.5/var, dtype='complex128') * tf.norm(y - H(fft(x_rec), psf))**2 )

    return cost_val



#@tf.function
def runADMM(observation, first_guess, psf, grad, z_0, mu_0, rho_0, rho_max, eta, gamma, 
            delta_0, ST_lambda, n_iter, alpha=1.0):
    
    cost = []
    nmse_arr = np.zeros(n_iter)
    x_opt = tf.zeros(tf.shape(observation.shape), dtype='complex128')
    count = 0
    
    # ADMM parameters
    x_k = first_guess
    z_k = z_0
    z_k1 = z_k
    mu_k = mu_0
    rho_k = rho_0         # Augmented lagrangian hyperparameter
    rho_k1 = rho_k
    delta_k = delta_0      
 
    # FISTA parameters
    t_k = tf.cast(1.0, dtype='complex128')
    z_fista_old = first_guess
    sigma = tf.cast(0.0, dtype='complex128')
    
        
    for k in range(n_iter):
        
        # FISTA update
        
        x_rec = z_fista_old - alpha * (grad(observation, z_fista_old, psf) + 
                                       mu_k * sigma**2 + 
                                       rho_k * sigma**2 * (z_fista_old - z_k))  
        
        x_rec = soft_thresh(x_rec, ST_lambda)
        x_rec = proj(x_rec)

        t_k1 = ( (tf.cast(1.0, dtype='complex128') + 
                  tf.math.sqrt(tf.cast(4.0, dtype='complex128')*t_k**2 + tf.cast(1.0, dtype='complex128'))) 
                / tf.cast(2.0, dtype='complex128') )
        
        lambda_fista = tf.cast(1.0, dtype='complex128') + (t_k - tf.cast(1.0, dtype='complex128')) / t_k1
        
        z_fista_new = x_k + lambda_fista * (x_rec - x_k)
        
        z_fista_old = z_fista_new
        t_k = t_k1
        
           
    
        # Denoising
        z_k1 = x_rec + mu_k/rho_k
        z_k1 = tf.cast(cv.GaussianBlur(np.abs(tf.keras.backend.get_value(z_k1)), (5,5), 0), dtype='complex128')
        
        # mu update
        mu_k1 = mu_k + (x_rec - z_k1)*rho_k
    
        
        # Cost
        cost.append(cost_func(observation, x_rec, z_k1, mu_k1, rho_k, psf))
        
        
        # Rho update
        delta_k1 = ( tf.cast(1.0/final_gal_tf.get_shape().as_list()[0], dtype='complex128') * 
                     (tf.norm(x_rec - x_k) + tf.norm(z_k1 - z_k) + tf.norm(mu_k1 - mu_k)) )
        
        
        if ( tf.keras.backend.get_value(tf.math.abs(delta_k1)) >= tf.keras.backend.get_value(tf.math.abs(eta * delta_k)) and  
             tf.keras.backend.get_value(tf.math.abs(rho_k1)) < tf.keras.backend.get_value(tf.math.abs(rho_max)) ):
            
            rho_k1 = gamma * rho_k
            
        else:
            rho_k1 = rho_k  
        
        
        x_k = x_rec
        mu_k = mu_k1
        z_k = z_k1
        rho_k = rho_k1
                
        
        # Convert back to signal domain
        final_im = tf.math.abs(fft(x_rec))
        nmse_arr[k] = nmse(gal_tf, tf.cast(final_im, dtype='complex128'))
        
        
        if (k!=0) and (nmse_arr[k]<nmse_arr[k-1]) and count==0:
            x_opt = final_im
        elif (k!=0) and (nmse_arr[k]>nmse_arr[k-1]):
            count += 1
        
        
        if k%200==0 and k!=0:
            
            fig = plt.subplots(1,3, figsize=(25,25))
            
            plt.subplot(131)
            plt.title('Reconstruction after {} iterations'.format(k))
            ax1 = plt.gca()
            im1 = ax1.imshow(tf.keras.backend.get_value(final_im))
            divider = make_axes_locatable(ax1)
            cax1 = divider.append_axes("right", size="5%", pad=0.05)
            plt.colorbar(im1, cax=cax1)

            plt.subplot(132)
            plt.title('Log Scale Intensity')
            plt.imshow(np.log10(tf.keras.backend.get_value(final_im)))

            plt.subplot(133)
            plt.title('Intensity Plot (Central Slice)')
            ax3 = plt.gca()
            ax3.plot(tf.keras.backend.get_value(final_im)[48,:], label='rec_im')           
            ax3.plot(tf.keras.backend.get_value(gal_tf)[48,:], label='gal')  
            plt.legend()
            ax3.set_aspect(1.0/ax3.get_data_ratio())
            
            plt.show(fig)
    
    return final_im, x_opt, nmse_arr, cost

In [ ]:
n_iter = 4000

# ADMM parameters (initial)
z_0 = tf.zeros(tf.shape(final_gal_tf), dtype='complex128')
mu_0 = tf.cast(0.0, dtype='complex128')
rho_0 = tf.cast(50.0, dtype='complex128')         # Augmented lagrangian hyperparameter
rho_max = tf.cast(200.0, dtype='complex128')      # max rho value
eta = tf.cast(0.5, dtype='complex128')            # should be between - [0, 1)
gamma = tf.cast(1.4, dtype='complex128')          # multiplicative rho factor -- Should be > 1
delta_0 = tf.cast(0.0, dtype='complex128')

first_guess = tf.zeros(tf.shape(final_gal_tf), dtype='complex128')


# Deconvolve the image
final_im, x_opt, nmse_arr, cost = runADMM(final_gal_tf, first_guess, psf_tf, grad, z_0, mu_0, rho_0, rho_max, 
                                          eta, gamma, delta_0, ST_lambda=1e-15, n_iter=n_iter)

In [ ]:
# Comparison

temp = tf.concat([x_opt, tf.math.real(gal_tf)], 1)

fig = plt.figure(figsize=(20,20))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 2]) 

ax1 = plt.subplot(gs[0])
plt.title('Observation')
ax1 = plt.gca()
im1 = ax1.imshow(tf.keras.backend.get_value(tf.math.abs(final_gal_tf)))
divider = make_axes_locatable(ax1)
cax1 = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im1, cax=cax1)

ax2 = plt.subplot(gs[1])
plt.title('Optimal Reconstruction vs. Ground Truth')
ax2 = plt.gca()
im2 = ax2.imshow(tf.keras.backend.get_value(temp))
ax2.axis('off')
divider = make_axes_locatable(ax2)
cax2 = divider.append_axes("right", size="2.5%", pad=0.05)
plt.colorbar(im2, cax=cax2)

plt.tight_layout()


fig2 = plt.subplots(1,3, figsize=(22,22))
plt.subplot(131)
plt.title('Residual (Y - H*X_est)')
ax1 = plt.gca()
im1 = ax1.imshow(tf.keras.backend.get_value(tf.math.abs(final_gal_tf - fftconvolve(psf_tf, tf.cast(x_opt, dtype='complex128')))))
divider = make_axes_locatable(ax1)
cax1 = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im1, cax=cax1)

plt.subplot(132)
plt.title('Log [FFT (Residual (Y - H*X_est))]')
ax2 = plt.gca()
im2 = ax2.imshow(tf.keras.backend.get_value(tf.math.log(tf.math.abs(fft(final_gal_tf - fftconvolve(psf_tf, tf.cast(x_opt, dtype='complex128')))))))
divider = make_axes_locatable(ax2)
cax2 = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im2, cax=cax2)

plt.subplot(133)
plt.title('Error (X_true - X_est)')
ax3 = plt.gca()
im3 = ax3.imshow(tf.keras.backend.get_value(x_opt - tf.math.real(gal_tf)))
divider = make_axes_locatable(ax3)
cax3 = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im3, cax=cax3)

plt.show(fig2)

In [ ]:
min_nmse = np.min(nmse_arr)
min_iter = np.where(nmse_arr == min_nmse)[0][0]

plt.figure(figsize=(16,8))
plt.plot(np.arange(1,n_iter+1), nmse_arr)
plt.title('NMSE vs. # Iterations')
plt.xlabel('Iterations')
plt.ylabel('NMSE')
plt.show()
print('\nMinimum NMSE = {} (at {} iterations)'.format(min_nmse, min_iter))

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(range(2500,3500), nmse_arr[2500:3500] )
plt.xlabel('# Iterations')
plt.ylabel('NMSE')
plt.show()